# Agent VectorDB 问答基准测试

这里我们介绍如何使用代理在多个向量数据库之间路由，对一个问答任务进行性能基准测试。

强烈建议您启用跟踪功能进行任何评估/基准测试。有关跟踪是什么以及如何设置的说明，请参见[此处](https://langchain.readthedocs.io/en/latest/tracing.html)。

In [1]:
# Comment this out if you are NOT using tracing
import os
os.environ["LANGCHAIN_HANDLER"] = "langchain"

## 加载数据
首先，让我们加载数据。

In [2]:
from langchain.evaluation.loading import load_dataset
dataset = load_dataset("agent-vectordb-qa-sota-pg")

Found cached dataset json (/Users/qt/.cache/huggingface/datasets/LangChainDatasets___json/LangChainDatasets--agent-vectordb-qa-sota-pg-d3ae24016b514f92/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)
100%|██████████| 1/1 [00:00<00:00, 414.42it/s]


In [3]:
dataset[0]

{'question': 'What is the purpose of the NATO Alliance?',
 'answer': 'The purpose of the NATO Alliance is to secure peace and stability in Europe after World War 2.',
 'steps': [{'tool': 'State of Union QA System', 'tool_input': None},
  {'tool': None, 'tool_input': 'What is the purpose of the NATO Alliance?'}]}

In [4]:
dataset[-1]

{'question': 'What is the purpose of YC?',
 'answer': 'The purpose of YC is to cause startups to be founded that would not otherwise have existed.',
 'steps': [{'tool': 'Paul Graham QA System', 'tool_input': None},
  {'tool': None, 'tool_input': 'What is the purpose of YC?'}]}

## 设置链
现在我们需要创建一些流水线来进行问答。第一步是在问题数据上创建索引。

In [5]:
from langchain.document_loaders import TextLoader
loader = TextLoader("../../modules/state_of_the_union.txt")

In [6]:
from langchain.indexes import VectorstoreIndexCreator

In [12]:
vectorstore_sota = VectorstoreIndexCreator(vectorstore_kwargs={"collection_name":"sota"}).from_loaders([loader]).vectorstore

Using embedded DuckDB without persistence: data will be transient


现在我们可以创建一个问答链。

In [13]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

In [16]:
chain_sota = RetrievalQA.from_chain_type(llm=OpenAI(temperature=0), chain_type="stuff", retriever=vectorstore_sota.as_retriever(), input_key="question")


类似地，我们可以为 Paul Graham 数据创建一个问答链。

In [17]:
loader = TextLoader("../../modules/paul_graham_essay.txt")

In [19]:
vectorstore_pg = VectorstoreIndexCreator(vectorstore_kwargs={"collection_name":"paul_graham"}).from_loaders([loader]).vectorstore

Using embedded DuckDB without persistence: data will be transient


In [20]:
chain_pg = RetrievalQA.from_chain_type(llm=OpenAI(temperature=0), chain_type="stuff", retriever=vectorstore_pg.as_retriever(), input_key="question")


我们现在可以设置一个代理来在它们之间进行路由。

In [22]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
tools = [
    Tool(
        name = "State of Union QA System",
        func=chain_sota.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question."
    ),
    Tool(
        name = "Paul Graham System",
        func=chain_pg.run,
        description="useful for when you need to answer questions about Paul Graham. Input should be a fully formed question."
    ),
]

In [34]:
agent = initialize_agent(tools, OpenAI(temperature=0), agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, max_iterations=4)

## 进行预测

首先，我们可以逐个数据点进行预测。以这种粒度进行操作允许我们详细地探索输出，并且比运行多个数据点更便宜。

In [35]:
agent.run(dataset[0]['question'])

'The purpose of the NATO Alliance is to secure peace and stability in Europe after World War 2.'

## 进行许多预测
现在我们可以进行预测。

In [36]:
predictions = []
predicted_dataset = []
error_dataset = []
for data in dataset:
    new_data = {"input": data["question"], "answer": data["answer"]}
    try:
        predictions.append(agent(new_data))
        predicted_dataset.append(new_data)
    except Exception:
        error_dataset.append(new_data)

## 评估表现
现在我们可以对预测进行评估。第一件事是可以通过肉眼查看它们。

In [37]:
predictions[0]

{'input': 'What is the purpose of the NATO Alliance?',
 'answer': 'The purpose of the NATO Alliance is to secure peace and stability in Europe after World War 2.',
 'output': 'The purpose of the NATO Alliance is to secure peace and stability in Europe after World War 2.'}

接下来，我们可以使用语言模型来以程序化的方式对它们进行评分。

In [38]:
from langchain.evaluation.qa import QAEvalChain

In [39]:
llm = OpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(predicted_dataset, predictions, question_key="input", prediction_key="output")

我们可以将经过评分的输出添加到`predictions`字典中，然后获取评分数量。

In [40]:
for i, prediction in enumerate(predictions):
    prediction['grade'] = graded_outputs[i]['text']

In [41]:
from collections import Counter
Counter([pred['grade'] for pred in predictions])

Counter({' CORRECT': 28, ' INCORRECT': 5})

我们还可以过滤掉错误示例的数据点并查看它们。

In [42]:
incorrect = [pred for pred in predictions if pred['grade'] == " INCORRECT"]

In [43]:
incorrect[0]

{'input': 'What are the four common sense steps that the author suggests to move forward safely?',
 'answer': 'The four common sense steps suggested by the author to move forward safely are: stay protected with vaccines and treatments, prepare for new variants, end the shutdown of schools and businesses, and stay vigilant.',
 'output': 'The four common sense steps suggested in the most recent State of the Union address are: cutting the cost of prescription drugs, providing a pathway to citizenship for Dreamers, revising laws so businesses have the workers they need and families don’t wait decades to reunite, and protecting access to health care and preserving a woman’s right to choose.',
 'grade': ' INCORRECT'}